In [10]:
import pandas as pd
import datetime

In [18]:
# import "calendar_statement_" + year + month + day + ".csv" with year month and day being the date of last sunday
# find the day  of the last sunday
today = datetime.date.today()
sunday = today - datetime.timedelta(days=today.weekday() + 1)
year = sunday.year
month = sunday.month
day = sunday.day
# import the file
# filename = "C:/Users/WilliamFetzner/AppData/Roaming/MetaQuotes/Terminal/0C2BFA140CA8FBEFEDCADDDEDD61AA24/MQL4/Files/calendar_statement_" + str(year) + '_' + str(month) + '_' +  str(day) + ".csv"
filename_path = "C:/Users/WilliamFetzner/AppData/Roaming/MetaQuotes/Terminal/0C2BFA140CA8FBEFEDCADDDEDD61AA24/tester/files/calendar_statement_"
for yr in [22, 23, 24]:
    for quarter in ['Jan_Mar', 'Apr_Jun', 'Jul_Sep', 'Oct_Dec']:
        if 'backtest_calendar_df' not in locals():
            backtest_calendar_df = pd.read_csv(f'{filename_path}{quarter}_{yr}.csv')
        else:
            try: 
                backtest_calendar_df = pd.concat([backtest_calendar_df, pd.read_csv(f'{filename_path}{quarter}_{yr}.csv')])
            except:
                break
backtest_calendar_df = backtest_calendar_df.drop_duplicates()

# filename_path

In [19]:
# reset the index
backtest_calendar_df = backtest_calendar_df.reset_index(drop=True)
backtest_calendar_df

,Date,Event,Impact,Currency
0,"2022, January 03, 10:15",(Spain) Markit Manufacturing PMI,High,EUR
1,"2022, January 03, 10:30",(Austria) Unemployment Rate,High,EUR
2,"2022, January 03, 10:45",(Italy) IHS Markit Manufacturing PMI,High,EUR
3,"2022, January 03, 10:55",(Germany) Markit Manufacturing PMI,High,EUR
4,"2022, January 03, 11:00",(Greece) Markit Manufacturing PMI,High,EUR
...,...,...,...,...
3194,"2024, March 29, 08:30",(United States) Personal Income MoM,High,USD
3195,"2024, March 29, 08:30",(United States) Personal Spending MoM,High,USD
3196,"2024, March 29, 08:30",(United States) Core PCE Price Index MoM,High,USD
3197,"2024, March 29, 08:30",(United States) Core PCE Price Index YoY,High,USD


In [20]:
# convert the date to datetime from format "YYYY, February 01, 00:00"
backtest_calendar_df['Date'] = pd.to_datetime(backtest_calendar_df['Date'], format='%Y, %B %d, %H:%M')
# adjust the datetime 7 hrs ahead to match market time
backtest_calendar_df['Date'] = backtest_calendar_df['Date'] + pd.Timedelta(hours=7)
backtest_calendar_df

,Date,Event,Impact,Currency
0,2022-01-03 17:15:00,(Spain) Markit Manufacturing PMI,High,EUR
1,2022-01-03 17:30:00,(Austria) Unemployment Rate,High,EUR
2,2022-01-03 17:45:00,(Italy) IHS Markit Manufacturing PMI,High,EUR
3,2022-01-03 17:55:00,(Germany) Markit Manufacturing PMI,High,EUR
4,2022-01-03 18:00:00,(Greece) Markit Manufacturing PMI,High,EUR
...,...,...,...,...
3194,2024-03-29 15:30:00,(United States) Personal Income MoM,High,USD
3195,2024-03-29 15:30:00,(United States) Personal Spending MoM,High,USD
3196,2024-03-29 15:30:00,(United States) Core PCE Price Index MoM,High,USD
3197,2024-03-29 15:30:00,(United States) Core PCE Price Index YoY,High,USD


In [21]:
# set the index to the date
# df_new_index = df.set_index('Date')
# create a new column for the week number of the year
backtest_calendar_df['Week'] = backtest_calendar_df['Date'].dt.isocalendar().week
# create a new column for year_week like 2024_01
backtest_calendar_df['Year_Week'] = backtest_calendar_df['Date'].dt.strftime('%Y_%W')
backtest_calendar_df

,Date,Event,Impact,Currency,Week,Year_Week
0,2022-01-03 17:15:00,(Spain) Markit Manufacturing PMI,High,EUR,1,2022_01
1,2022-01-03 17:30:00,(Austria) Unemployment Rate,High,EUR,1,2022_01
2,2022-01-03 17:45:00,(Italy) IHS Markit Manufacturing PMI,High,EUR,1,2022_01
3,2022-01-03 17:55:00,(Germany) Markit Manufacturing PMI,High,EUR,1,2022_01
4,2022-01-03 18:00:00,(Greece) Markit Manufacturing PMI,High,EUR,1,2022_01
...,...,...,...,...,...,...
3194,2024-03-29 15:30:00,(United States) Personal Income MoM,High,USD,13,2024_13
3195,2024-03-29 15:30:00,(United States) Personal Spending MoM,High,USD,13,2024_13
3196,2024-03-29 15:30:00,(United States) Core PCE Price Index MoM,High,USD,13,2024_13
3197,2024-03-29 15:30:00,(United States) Core PCE Price Index YoY,High,USD,13,2024_13


In [24]:
# are there any weeks missing? 
# resample the dataframe by week and count the number of events
# set the index to 'Date'
backtest_calendar_df_indexed = backtest_calendar_df.set_index('Date')
calendar_df_weekly = backtest_calendar_df_indexed.resample('W').agg({'Event': 'count'})
calendar_df_weekly[calendar_df_weekly['Event'] == 0]

,Event
Date,
2022-01-09,44
2022-01-16,17
2022-01-23,10
2022-01-30,35
2022-02-06,50
...,...
2024-03-03,53
2024-03-10,27
2024-03-17,23


In [25]:
backtest_calendar_df.sort_values(by='Date', ascending=True)

,Date,Event,Impact,Currency,Week,Year_Week
0,2022-01-03 17:15:00,(Spain) Markit Manufacturing PMI,High,EUR,1,2022_01
1,2022-01-03 17:30:00,(Austria) Unemployment Rate,High,EUR,1,2022_01
2,2022-01-03 17:45:00,(Italy) IHS Markit Manufacturing PMI,High,EUR,1,2022_01
3,2022-01-03 17:55:00,(Germany) Markit Manufacturing PMI,High,EUR,1,2022_01
4,2022-01-03 18:00:00,(Greece) Markit Manufacturing PMI,High,EUR,1,2022_01
...,...,...,...,...,...,...
3195,2024-03-29 15:30:00,(United States) Personal Spending MoM,High,USD,13,2024_13
3197,2024-03-29 15:30:00,(United States) Core PCE Price Index YoY,High,USD,13,2024_13
3193,2024-03-29 15:30:00,(United States) Goods Trade Balance,High,USD,13,2024_13
3194,2024-03-29 15:30:00,(United States) Personal Income MoM,High,USD,13,2024_13


In [27]:
for i in backtest_calendar_df.Year_Week.unique():
    df_wk = backtest_calendar_df[backtest_calendar_df['Year_Week'] == i]
    # sort the dataframe by date
    df_wk = df_wk.sort_values(by='Date', ascending=True)
    # grab the date of the first day of the week
    first_day = df_wk['Date'].iloc[0]
    # find out what the previous sunday was for first_day
    sunday = first_day - datetime.timedelta(days=first_day.weekday() + 1)
    year = sunday.year
    month = sunday.month
    day = sunday.day
    # get the filename ready
    filename = f"{filename_path}{year}_{month}_{day}.csv"
    # print(filename)
    # save the file
    df_wk.to_csv(filename, index=False)

    